In [13]:
import pandas as pd
import datetime
import copy

### 波动率曲面数据VolatilitySurface1996_2020

In [14]:
#清理好的数据为vol_clean_df，vol_clean_df_c，vol_clean_df_p
path = 'D:\Desktop\RND\VolatilitySurface1996_2020.csv'
vol_df = pd.read_csv(
    path,
    usecols=['date', 'days', 'impl_volatility', 'impl_strike', 'cp_flag'])

In [15]:
vol_clean_df = vol_df.dropna()
vol_clean_df = vol_clean_df.reset_index(drop=True)
# 处理时间，变成可以用时间加减的年月日
vol_clean_df['time_date'] = vol_clean_df['date'].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'))

In [16]:
print(vol_clean_df.head(10))

       date  days  impl_volatility  impl_strike cp_flag  time_date
0  19960104    30         0.138704     652.5124       P 1996-01-04
1  19960104    30         0.137368     645.8368       P 1996-01-04
2  19960104    30         0.135625     640.5763       P 1996-01-04
3  19960104    30         0.132932     636.0494       P 1996-01-04
4  19960104    30         0.128539     631.9539       P 1996-01-04
5  19960104    30         0.124021     628.3651       P 1996-01-04
6  19960104    30         0.121505     625.2828       P 1996-01-04
7  19960104    30         0.120634     622.4753       P 1996-01-04
8  19960104    30         0.121280     619.7742       P 1996-01-04
9  19960104    30         0.123523     617.0321       P 1996-01-04


In [17]:
print(vol_clean_df['days'].value_counts())
print(vol_clean_df['date'].unique())
print(vol_clean_df['date'].value_counts())
print(len(vol_clean_df['date'].unique()))  #有6293个不同的开始时间
print(vol_clean_df['cp_flag'].value_counts())
vol_clean_df_c = vol_clean_df[vol_clean_df['cp_flag'] == 'C']  #选出看涨
vol_clean_df_p = vol_clean_df[vol_clean_df['cp_flag'] == 'P']  #选出看跌

30     213962
60     213962
91     213962
122    213962
152    213962
182    213962
273    213962
365    213962
547    213962
730    213962
10      95914
Name: days, dtype: int64
[19960104 19960105 19960108 ... 20201229 20201230 20201231]
20080703    374
20130918    374
20130930    374
20130927    374
20130926    374
           ... 
20111024    340
20111025    340
20111026    340
20041203    340
19960104    340
Name: date, Length: 6293, dtype: int64
6293
P    1117767
C    1117767
Name: cp_flag, dtype: int64


In [7]:
print(len(vol_clean_df_c))

1117767


### 利率数据InterestRate1996_2020

In [8]:
#清理好的全部数据为rate_clean_df
rate_path = 'D:\Desktop\RND\InterestRate1996_2020.csv'
rate_df = pd.read_csv(rate_path)
rate_clean_df = rate_df.dropna()
rate_clean_df['time_date'] = rate_df['date'].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'))
print(rate_clean_df.head(10))

       date  days      rate  time_date
0  19960102     9  5.763067 1996-01-02
1  19960102    15  5.745902 1996-01-02
2  19960102    50  5.673317 1996-01-02
3  19960102    78  5.608884 1996-01-02
4  19960102   169  5.473762 1996-01-02
5  19960102   260  5.352667 1996-01-02
6  19960102   351  5.276194 1996-01-02
7  19960102   442  5.249759 1996-01-02
8  19960102   533  5.235769 1996-01-02
9  19960102   624  5.243372 1996-01-02


In [9]:
rate_clean_df[rate_clean_df['time_date'] ==
              vol_clean_df_c['time_date'].iloc[0]].head(5)

,date,days,rate,time_date
86,19960104,11,5.729460,1996-01-04
87,19960104,13,5.726104,1996-01-04
88,19960104,48,5.664931,1996-01-04
89,19960104,76,5.601891,1996-01-04
90,19960104,167,5.468961,1996-01-04


In [10]:
# 实验：给部分的vol_c的数据添加上对应的rate数据
index = list(range(0,30000,300))
vol_c_part = vol_clean_df_c.iloc[index].reset_index(drop=True)
print((vol_c_part.head(10)))

       date  days  impl_volatility  impl_strike cp_flag  time_date
0  19960104    30         0.099845     642.7291       C 1996-01-04
1  19960105   365         0.138055     603.2400       C 1996-01-05
2  19960109   182         0.111855     639.1638       C 1996-01-09
3  19960111    91         0.163770     547.1875       C 1996-01-11
4  19960115    30         0.137284     595.7184       C 1996-01-15
5  19960116   547         0.131422     691.2406       C 1996-01-16
6  19960118   182         0.168007     545.4205       C 1996-01-18
7  19960122   122         0.119582     614.4843       C 1996-01-22
8  19960124    60         0.100903     640.6282       C 1996-01-24
9  19960125   547         0.182849     526.7268       C 1996-01-25


In [11]:
#实验：对vol数据取出来一部分尝试讲rate值对应（插值）进去
vol_c_part_rate = []
for index_vol in range(vol_c_part.shape[0]):
    time = vol_c_part.iloc[index_vol]['time_date']
    day = vol_c_part.iloc[index_vol]['days']
    rate_clean_df_time = rate_clean_df[rate_clean_df['time_date'] == time].reset_index(drop=True)     
    if rate_clean_df_time[rate_clean_df_time['days'] == day].shape[0] > 0:
        vol_c_part_rate.append(rate_clean_df_time[rate_clean_df_time['days'] == day]['rate'].iloc[0])
    else:
        rate_clean_df_time['cal_day'] = abs(rate_clean_df_time['days']-day)  
        min_index = []
        for _ in range(2):
            index_now = rate_clean_df_time['cal_day'].idxmin()
            rate_clean_df_time.loc[index_now,'cal_day'] = 100
            min_index.append(index_now)
        x_1 = rate_clean_df_time['days'].iloc[min_index[0]]
        x_2 = rate_clean_df_time['days'].iloc[min_index[1]]
        y_1 = rate_clean_df_time['rate'].iloc[min_index[0]]
        y_2 = rate_clean_df_time['rate'].iloc[min_index[1]]
        vol_c_part_rate.append(y_1 + (day-x_1)/(x_2-x_1)*(y_2-y_1))
        
vol_c_part['rate'] = vol_c_part_rate
print(vol_c_part.head(5))

       date  days  impl_volatility  impl_strike cp_flag  time_date      rate
0  19960104    30         0.099845     642.7291       C 1996-01-04  5.696391
1  19960105   365         0.138055     603.2400       C 1996-01-05  5.267515
2  19960109   182         0.111855     639.1638       C 1996-01-09  5.456646
3  19960111    91         0.163770     547.1875       C 1996-01-11  5.568398
4  19960115    30         0.137284     595.7184       C 1996-01-15  5.689823


In [13]:
#test
a = [1, 3, 6, 7, 9, 11, 13]
#aa = [i+1 for i in a]
b = pd.DataFrame(a,columns=['data'])
b['new'] = [abs(i-5) for i in a]
b['rate'] = [i*2 for i in a]
bb = copy.deepcopy(b)
#print(b['data'])
#print(abs(b-4))

min_number = []
min_index = []

for _ in range(2):
    index_now = bb['new'].idxmin()
    number = bb['data'].iloc[index_now]

    bb.loc[index_now,'new'] = 100
    min_number.append(number)
    min_index.append(index_now)   
print(min_index)
print(min_number)

x_1 = b['data'].iloc[min_index[0]]
x_2 = b['data'].iloc[min_index[1]]
y_1 = b['rate'].iloc[min_index[0]]
y_2 = b['rate'].iloc[min_index[1]]
print(y_1 + (5-x_1)/(x_2-x_1)*(y_2-y_1))  
#a.append(b[b['data']==1]['neww'].iloc[0])
#print(a)
#b[b['data']==1]['neww'].iloc[0]

[2, 1]
[6, 3]
10.0


### 标的价格st IndexPrice1996_2020

In [18]:
#清理好的全部数据在st_df
st_path = 'D:\Desktop\RND\IndexPrice1996_2020.csv'
st_df = pd.read_csv(st_path, usecols=['date', 'open', 'close'])
st_df['St'] = (st_df['open'] + st_df['open']) / 2
st_df = st_df.dropna()
st_df = st_df.reset_index(drop=True)
st_df['time_date'] = st_df['date'].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'))
print(st_df.head(5))

       date    open   close      St  time_date
0  19960102  615.93  620.73  615.93 1996-01-02
1  19960103  620.73  621.32  620.73 1996-01-03
2  19960104  621.32  617.70  621.32 1996-01-04
3  19960105  617.70  616.71  617.70 1996-01-05
4  19960108  616.71  618.46  616.71 1996-01-08


### 期权价格数据SP500OptionPrice1996_2020_2

In [19]:
#清理好的全部数据在option_df（含有st和rate，不含1中vol的days和iv）
optionPrice_path = 'D:\Desktop\RND\SP500OptionPrice1996_2020_2.csv'
option_df = pd.read_csv(optionPrice_path
                        ,usecols=['date','exdate','cp_flag', 'strike_price'
                                  ,'impl_volatility', 'delta','gamma','theta', 'vega'
                                  , 'best_bid', 'best_offer', 'volume'] )
option_df = option_df.dropna()
option_df = option_df.reset_index(drop=True)
option_df['strike_price'] = option_df['strike_price']/1000
option_df['price'] = (option_df['best_bid'] + option_df['best_offer'])/2
option_df['time_date'] = option_df['date'].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'))
option_df['time_exdate'] = option_df['exdate'].apply(
    lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'))
option_df['days'] = (option_df['time_exdate']-option_df['time_date'])
option_df['days'] = option_df['days'].astype('str').apply(lambda x:x[:-5]).astype('int64')
option_df['tau'] = option_df['days']/360

option_df['St'] = [st_df[st_df['date']==date]['St'].iloc[0] for date in option_df['date']]

In [16]:
#所有option data中加入rate数据（插值）太大了不好跑，之后先按年份分离再添加rate
option_part_rate = []
for index_vol in range(option_df.shape[0]):
    time = option_df.iloc[index_vol]['time_date']
    day = option_df.iloc[index_vol]['days']
    rate_clean_df_time = rate_clean_df[rate_clean_df['time_date'] == time].reset_index(drop=True)     
    if rate_clean_df_time[rate_clean_df_time['days'] == day].shape[0] > 0:
        option_part_rate.append(rate_clean_df_time[rate_clean_df_time['days'] == day]['rate'].iloc[0])
    elif rate_clean_df_time.shape[0]==0:
        option_part_rate.append(np.nan)
    else:
        rate_clean_df_time['cal_day'] = abs(rate_clean_df_time['days']-day)  
        min_index = []
        for _ in range(2):
            index_now = rate_clean_df_time['cal_day'].idxmin()
            rate_clean_df_time.loc[index_now,'cal_day'] = 100
            min_index.append(index_now)
        x_1 = rate_clean_df_time['days'].iloc[min_index[0]]
        x_2 = rate_clean_df_time['days'].iloc[min_index[1]]
        y_1 = rate_clean_df_time['rate'].iloc[min_index[0]]
        y_2 = rate_clean_df_time['rate'].iloc[min_index[1]]
        option_part_rate.append(y_1 + (day-x_1)/(x_2-x_1)*(y_2-y_1))
        
option_df['rate'] = option_part_rate
print(option_df.head(5))
#print(option_part_df.isna().sum())

KeyboardInterrupt: 

In [19]:
# 实验：给部分的optin价格的数据添加上对应的st，rate，vol数据中最相近（或插值）的day和对应的vol（虽然在option数据中有IV的数据，两个可以做对比）   
index_option_part = list(range(0,300000,3000))
option_part_df = option_df.iloc[index_option_part].reset_index(drop=True)
print((option_part_df.head(5)))
print(option_part_df.shape[0])

       date    exdate cp_flag  strike_price  best_bid  best_offer  volume  \
0  20080609  20080613       C        1250.0     110.0       116.0       0   
1  20080611  20090620       C        1100.0     286.1       290.1       0   
2  20080616  20101218       C        2100.0       9.3        12.8       0   
3  20080619  20080816       C        1460.0       3.2         4.0      53   
4  20080624  20090620       P        1350.0     119.6       123.5       1   

        vega   price  time_date time_exdate  days       tau       St  
0   12.92499  113.00 2008-06-09  2008-06-13     4  0.011111  1360.83  
1  355.87810  288.10 2008-06-11  2009-06-20   374  1.038889  1357.09  
2  310.70210   11.05 2008-06-16  2010-12-18   915  2.541667  1358.85  
3   88.85300    3.60 2008-06-19  2008-08-16    58  0.161111  1336.89  
4  508.78520  121.55 2008-06-24  2009-06-20   361  1.002778  1317.23  
100


In [21]:
#实验：option data中加入rate数据（插值）
option_part_rate = []
for index_vol in range(option_part_df.shape[0]):
    time = option_part_df.iloc[index_vol]['time_date']
    day = option_part_df.iloc[index_vol]['days']
    rate_clean_df_time = rate_clean_df[rate_clean_df['time_date'] == time].reset_index(drop=True)     
    if rate_clean_df_time[rate_clean_df_time['days'] == day].shape[0] > 0:
        option_part_rate.append(rate_clean_df_time[rate_clean_df_time['days'] == day]['rate'].iloc[0])
    if rate_clean_df_time.shape[0]==0:
        option_part_rate.append(np.nan)
    else:
        rate_clean_df_time['cal_day'] = abs(rate_clean_df_time['days']-day)  
        min_index = []
        for _ in range(2):
            index_now = rate_clean_df_time['cal_day'].idxmin()
            rate_clean_df_time.loc[index_now,'cal_day'] = 100
            min_index.append(index_now)
        x_1 = rate_clean_df_time['days'].iloc[min_index[0]]
        x_2 = rate_clean_df_time['days'].iloc[min_index[1]]
        y_1 = rate_clean_df_time['rate'].iloc[min_index[0]]
        y_2 = rate_clean_df_time['rate'].iloc[min_index[1]]
        option_part_rate.append(y_1 + (day-x_1)/(x_2-x_1)*(y_2-y_1))
        
option_part_df['rate'] = option_part_rate
print(option_part_df.head(5))
print(option_part_df['rate'].isna().sum())

       date    exdate cp_flag  strike_price  best_bid  best_offer  volume  \
0  20080609  20080613       C        1250.0     110.0       116.0       0   
1  20080611  20090620       C        1100.0     286.1       290.1       0   
2  20080616  20101218       C        2100.0       9.3        12.8       0   
3  20080619  20080816       C        1460.0       3.2         4.0      53   
4  20080624  20090620       P        1350.0     119.6       123.5       1   

        vega   price  time_date time_exdate  days       tau       St      rate  
0   12.92499  113.00 2008-06-09  2008-06-13     4  0.011111  1360.83  2.323277  
1  355.87810  288.10 2008-06-11  2009-06-20   374  1.038889  1357.09  3.335485  
2  310.70210   11.05 2008-06-16  2010-12-18   915  2.541667  1358.85  4.194371  
3   88.85300    3.60 2008-06-19  2008-08-16    58  0.161111  1336.89  2.755788  
4  508.78520  121.55 2008-06-24  2009-06-20   361  1.002778  1317.23  3.332439  
1


### 存储下来清理好的optiondata，（含St和rate和年份分组标签，在RND文件夹里）

In [20]:
#加上一列‘年份’属于的年份，之后可以用来做选取
print(option_df['date'].min())
print(option_df['date'].max())
bins = list(range(20080000,20220000,10000)) #是bins,'三个端点两个区间'
labels = list(range(2008,2021,1)) #'两个'，是labels，区间名
option_df['nianfen'] = pd.cut(option_df['date'], bins=bins,right=False,labels=labels)   
option_df.to_csv(r'D:\Desktop\RND\option_clean_all_df.csv', index = False)

20080609
20201231


In [21]:
#读取之前存取的处理好的optiondata（2008-2020年）
# option_df_read和上面的option_df完全一样，包含所有的option数据（包括St和rate）
option_df_read = pd.read_csv('D:\Desktop\RND\option_clean_all_df.csv',
                             usecols=[
                                 'date', 'exdate', 'time_date', 'time_exdate',
                                 'cp_flag', 'strike_price', 'impl_volatility', 
                                 'volume','delta','gamma','theta', 'vega',
                                 'price', 'days', 'tau', 'St', 'nianfen'
                             ])
print(option_df_read.shape)
print(option_df.shape)

(19187758, 17)
(19187758, 19)


In [50]:
#之前忘记加iv列的
option_df_read['nianfen'].value_counts() 

2020    3860513
2019    3135258
2018    2878953
2017    2053488
2015    1727372
2016    1643671
2014    1120628
2013     728362
2012     577224
2011     458767
2010     428025
2009     391822
2008     183675
Name: nianfen, dtype: int64

In [22]:
#新加上iv数据的
option_df_read['nianfen'].value_counts()

2020    3860513
2019    3135258
2018    2878953
2017    2053488
2015    1727372
2016    1643671
2014    1120628
2013     728362
2012     577224
2011     458767
2010     428025
2009     391822
2008     183675
Name: nianfen, dtype: int64

In [23]:
option_df_2020 = option_df_read[option_df_read['nianfen']==2020].reset_index(drop=True)  
option_df_2019 = option_df_read[option_df_read['nianfen']==2019].reset_index(drop=True)  
option_df_2018 = option_df_read[option_df_read['nianfen']==2018].reset_index(drop=True)  


In [14]:
#所有option data中加入rate数据（插值）太大了不好跑，之后先按年份分离再添加rate
#（还是跑不出来）
option_part_rate = []
for index_vol in range(option_df_2020.shape[0]):
    time = option_df_2020.iloc[index_vol]['time_date']
    day = option_df_2020.iloc[index_vol]['days']
    rate_clean_df_time = rate_clean_df[rate_clean_df['time_date'] == time].reset_index(drop=True)     
    if rate_clean_df_time[rate_clean_df_time['days'] == day].shape[0] > 0:
        option_part_rate.append(rate_clean_df_time[rate_clean_df_time['days'] == day]['rate'].iloc[0])
    if rate_clean_df_time.shape[0]==0:
        option_part_rate.append(np.nan)
    else:
        rate_clean_df_time['cal_day'] = abs(rate_clean_df_time['days']-day)  
        min_index = []
        for _ in range(2):
            index_now = rate_clean_df_time['cal_day'].idxmin()
            rate_clean_df_time.loc[index_now,'cal_day'] = 100
            min_index.append(index_now)
        x_1 = rate_clean_df_time['days'].iloc[min_index[0]]
        x_2 = rate_clean_df_time['days'].iloc[min_index[1]]
        y_1 = rate_clean_df_time['rate'].iloc[min_index[0]]
        y_2 = rate_clean_df_time['rate'].iloc[min_index[1]]
        option_part_rate.append(y_1 + (day-x_1)/(x_2-x_1)*(y_2-y_1))
        
option_df_2020['rate'] = option_part_rate
print(option_df_2020.head(5))

KeyboardInterrupt: 

In [ ]:
#所有option data中加入rate数据（插值）太大了不好跑，之后先按年份分离再添加rate
#（还是跑不出来）
option_part_rate = []
for index_vol in range(option_df_2019.shape[0]):
    time = option_df_2019.iloc[index_vol]['time_date']
    day = option_df_2019.iloc[index_vol]['days']
    rate_clean_df_time = rate_clean_df[rate_clean_df['time_date'] == time].reset_index(drop=True)     
    if rate_clean_df_time[rate_clean_df_time['days'] == day].shape[0] > 0:
        option_part_rate.append(rate_clean_df_time[rate_clean_df_time['days'] == day]['rate'].iloc[0])
    if rate_clean_df_time.shape[0]==0:
        option_part_rate.append(np.nan)
    else:
        rate_clean_df_time['cal_day'] = abs(rate_clean_df_time['days']-day)  
        min_index = []
        for _ in range(2):
            index_now = rate_clean_df_time['cal_day'].idxmin()
            rate_clean_df_time.loc[index_now,'cal_day'] = 100
            min_index.append(index_now)
        x_1 = rate_clean_df_time['days'].iloc[min_index[0]]
        x_2 = rate_clean_df_time['days'].iloc[min_index[1]]
        y_1 = rate_clean_df_time['rate'].iloc[min_index[0]]
        y_2 = rate_clean_df_time['rate'].iloc[min_index[1]]
        option_part_rate.append(y_1 + (day-x_1)/(x_2-x_1)*(y_2-y_1))
        
option_df_2019['rate'] = option_part_rate
print(option_df_2019.head(5))

In [ ]:
#所有option data中加入rate数据（插值）太大了不好跑，之后先按年份分离再添加rate
#（还是跑不出来）
option_part_rate = []
for index_vol in range(option_df_2018.shape[0]):
    time = option_df_2018.iloc[index_vol]['time_date']
    day = option_df_2018.iloc[index_vol]['days']
    rate_clean_df_time = rate_clean_df[rate_clean_df['time_date'] == time].reset_index(drop=True)     
    if rate_clean_df_time[rate_clean_df_time['days'] == day].shape[0] > 0:
        option_part_rate.append(rate_clean_df_time[rate_clean_df_time['days'] == day]['rate'].iloc[0])
    if rate_clean_df_time.shape[0]==0:
        option_part_rate.append(np.nan)
    else:
        rate_clean_df_time['cal_day'] = abs(rate_clean_df_time['days']-day)  
        min_index = []
        for _ in range(2):
            index_now = rate_clean_df_time['cal_day'].idxmin()
            rate_clean_df_time.loc[index_now,'cal_day'] = 100
            min_index.append(index_now)
        x_1 = rate_clean_df_time['days'].iloc[min_index[0]]
        x_2 = rate_clean_df_time['days'].iloc[min_index[1]]
        y_1 = rate_clean_df_time['rate'].iloc[min_index[0]]
        y_2 = rate_clean_df_time['rate'].iloc[min_index[1]]
        option_part_rate.append(y_1 + (day-x_1)/(x_2-x_1)*(y_2-y_1))
        
option_df_2018['rate'] = option_part_rate
print(option_df_2018.head(5))

In [24]:
option_df_2020.to_csv(r'D:\Desktop\RND\option_clean_all_2020_df.csv', index = False)
option_df_2019.to_csv(r'D:\Desktop\RND\option_clean_all_2019_df.csv', index = False)
option_df_2018.to_csv(r'D:\Desktop\RND\option_clean_all_2018_df.csv', index = False)
print(option_df_2020.head(5))

       date    exdate cp_flag  strike_price  volume  impl_volatility  \
0  20200102  20200117       C        2860.0       0         0.186688   
1  20200102  20200117       C        2865.0       0         0.173579   
2  20200102  20200117       C        2885.0       0         0.203486   
3  20200102  20200117       C        2890.0       0         0.215346   
4  20200102  20200117       C        2895.0       0         0.211763   

      delta     gamma      vega      theta   price   time_date time_exdate  \
0  0.999147  0.000006  0.420252   9.536298  397.45  2020-01-02  2020-01-17   
1  0.999245  0.000003  0.189318  10.048300  392.45  2020-01-02  2020-01-17   
2  0.998244  0.000028  2.294910   4.055100  372.50  2020-01-02  2020-01-17   
3  0.997207  0.000048  4.249265  -1.871995  367.55  2020-01-02  2020-01-17   
4  0.997247  0.000048  4.176572  -1.555401  362.55  2020-01-02  2020-01-17   

   days       tau       St  nianfen  
0    15  0.041667  3244.67     2020  
1    15  0.041667  324

In [25]:
#以后用的时候可以直接read(2020,2019,2018这三年的数据里不包含rate)
option_df_2020_read = pd.read_csv('D:\Desktop\RND\option_clean_all_2020_df.csv',
                             usecols=[
                                 'date', 'exdate', 'time_date', 'time_exdate',
                                 'cp_flag', 'strike_price', 'impl_volatility', 
                                 'volume','delta','gamma','theta', 'vega',
                                 'price', 'days', 'tau', 'St', 'nianfen'
                             ])

option_df_2019_read = pd.read_csv('D:\Desktop\RND\option_clean_all_2019_df.csv',
                             usecols=[
                                 'date', 'exdate', 'time_date', 'time_exdate',
                                 'cp_flag', 'strike_price', 'impl_volatility', 
                                 'volume','delta','gamma','theta', 'vega',
                                 'price', 'days', 'tau', 'St', 'nianfen'
                             ])
option_df_2018_read = pd.read_csv('D:\Desktop\RND\option_clean_all_2018_df.csv',
                             usecols=[
                                 'date', 'exdate', 'time_date', 'time_exdate',
                                 'cp_flag', 'strike_price', 'impl_volatility', 
                                 'volume','delta','gamma','theta', 'vega',
                                 'price', 'days', 'tau', 'St', 'nianfen'
                             ])
print(option_df_2020_read.shape)
print(option_df_2019_read.shape)
print(option_df_2018_read.shape)

(3860513, 17)
(3135258, 17)
(2878953, 17)


### 读取完上面分离出来的不带rate的2020年数据

In [26]:
print(option_df_2020_read.head(5))

       date    exdate cp_flag  strike_price  volume  impl_volatility  \
0  20200102  20200117       C        2860.0       0         0.186688   
1  20200102  20200117       C        2865.0       0         0.173579   
2  20200102  20200117       C        2885.0       0         0.203486   
3  20200102  20200117       C        2890.0       0         0.215346   
4  20200102  20200117       C        2895.0       0         0.211763   

      delta     gamma      vega      theta   price   time_date time_exdate  \
0  0.999147  0.000006  0.420252   9.536298  397.45  2020-01-02  2020-01-17   
1  0.999245  0.000003  0.189318  10.048300  392.45  2020-01-02  2020-01-17   
2  0.998244  0.000028  2.294910   4.055100  372.50  2020-01-02  2020-01-17   
3  0.997207  0.000048  4.249265  -1.871995  367.55  2020-01-02  2020-01-17   
4  0.997247  0.000048  4.176572  -1.555401  362.55  2020-01-02  2020-01-17   

   days       tau       St  nianfen  
0    15  0.041667  3244.67     2020  
1    15  0.041667  324

In [27]:
option_df_2020_read_c = option_df_2020_read[option_df_2020_read['cp_flag']=='C'] 
option_df_2020_read_p = option_df_2020_read[option_df_2020_read['cp_flag']=='P'] 

#分离开出2020年1月的数据，看涨，有十万数据
option_df_2020_read_c_1 = option_df_2020_read_c[option_df_2020_read_c['date']<20200200]  
print((option_df_2020_read_c_1.shape))

(152416, 17)


In [28]:
#查数量
value_count_2020_1 = option_df_2020_read_c_1.groupby(
    ['date', 'days'])['strike_price'].count()

In [29]:
value_count_2020_1_df = pd.DataFrame(value_count_2020_1)
value_count_2020_1_df.to_csv(r'D:\Desktop\RND\value_count_2020_1_df.csv', index = True)
print(value_count_2020_1_df)

               strike_price
date     days              
20200102 1               59
         4               57
         6               64
         8               82
         11              81
...                     ...
20200131 350            103
         413            102
         504            107
         686            117
         1050           105

[829 rows x 1 columns]


In [30]:
option_df_2020_read_c_1.groupby('date').count()  #按开始日看期权数量

,exdate,cp_flag,strike_price,volume,impl_volatility,delta,gamma,vega,theta,price,time_date,time_exdate,days,tau,St,nianfen
date,,,,,,,,,,,,,,,,
20200102,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133
20200103,7958,7958,7958,7958,7958,7958,7958,7958,7958,7958,7958,7958,7958,7958,7958,7958
20200106,6922,6922,6922,6922,6922,6922,6922,6922,6922,6922,6922,6922,6922,6922,6922,6922
20200107,7910,7910,7910,7910,7910,7910,7910,7910,7910,7910,7910,7910,7910,7910,7910,7910
20200108,7775,7775,7775,7775,7775,7775,7775,7775,7775,7775,7775,7775,7775,7775,7775,7775
20200109,7459,7459,7459,7459,7459,7459,7459,7459,7459,7459,7459,7459,7459,7459,7459,7459
20200110,8119,8119,8119,8119,8119,8119,8119,8119,8119,8119,8119,8119,8119,8119,8119,8119
20200113,6759,6759,6759,6759,6759,6759,6759,6759,6759,6759,6759,6759,6759,6759,6759,6759
20200114,6710,6710,6710,6710,6710,6710,6710,6710,6710,6710,6710,6710,6710,6710,6710,6710
